## Tools

In [2]:
import pandas as pd

In [58]:
from datetime import datetime
from typing import Literal

def natural_date_to_ms(date_str:str)->int:
    
    #fmt = "%Y-%m-%d %H:%M"
    fmt = "%Y-%m-%d"

    # Conversion en datetime
    dt = datetime.strptime(date_str, fmt)

    # Conversion en millisecondes
    return int(dt.timestamp() * 1000)

   
def annualized_8h(rate_8h:float, type: Literal["linear", "compound"] = "linear"):
    # Return directly the annualised rate as percentage
    if type =="compound":
        return ((1 + rate_8h/100) ** 1095 - 1)*100 
    return rate_8h * 1095 *100 #24 / 8 * 365


## Binance

### Historic Binance Rate

In [5]:
from binance.cm_futures import CMFutures
import logging
from binance.lib.utils import config_logging


def get_binance_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    if start_ms:
        start_ms = natural_date_to_ms(start_ms)
    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
        assert(start_ms < end_ms),"End date must greater than Start Date"


    config_logging(logging, logging.DEBUG)

    cm_futures_client = CMFutures()
    temp=cm_futures_client.funding_rate(coin, **{"limit": 1000,"startTime": start_ms, "endTime": end_ms})
    #temp=cm_futures_client.funding_rate("BTCUSD_PERP", **{"limit": 1000})

    df = pd.DataFrame(temp)
    del(temp)
    # Convert types
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = pd.to_numeric(df['fundingRate'])
    df['markPrice'] = pd.to_numeric(df['markPrice'])
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop(columns=["symbol","fundingRate"])
    .drop_duplicates()
    .set_index('fundingTime')
    .rename(columns={'annualized_funding': 'annualized_funding_binance'})
    )
    df.index = df.index.floor('min')  # 'T' is shorthand for 'min'

    return df

## Bybit

### Historic Bybit Rate

In [8]:
import pandas as pd
from pybit.unified_trading import HTTP
import time

def get_bybit_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
    if start_ms:
        start_ms = natural_date_to_ms(start_ms)
        assert(start_ms < end_ms),"End date must greater than Start Date"
        # faire le test pour verifier que l'on ne prends pas plus de 200 valeurs dans l'interval

    
    session = HTTP()
    limit = 100
    all_data = []

    current_start = start_ms
    while True:
        temp = session.get_funding_rate_history(
            category="linear",
            symbol=coin,
            startTime=current_start,
            endTime=current_start + 5760000000,
            limit=limit
        )

        result = temp["result"]["list"]
        if not result:
            break
        
        all_data.extend(result)

        # Stop if less than limit entries returned (means last page)
        if len(result) < limit:
            break

        # Move to next time window: set current_start to 1 ms after last returned timestamp
        last_timestamp = int(result[-1]['fundingRateTimestamp'])
        current_start = last_timestamp + 1

        time.sleep(0.2)  # Respect API rate limit

    df = pd.DataFrame(all_data)
    if df.empty:
        return df

    df['fundingRateTimestamp'] = pd.to_numeric(df['fundingRateTimestamp'])
    df['fundingRateTimestamp'] = pd.to_datetime(df['fundingRateTimestamp'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding'] = df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop_duplicates()
    .set_index('fundingRateTimestamp')
    .rename(columns={'annualized_funding': 'annualized_funding_bybit'})
    )
    df.index = df.index.floor('min')  # type: ignore # 'T' is shorthand for 'min'

    return df


In [ ]:
from pybit.unified_trading import HTTP

def get_dft_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    # error if only start time

    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
        if start_ms:
            start_ms = natural_date_to_ms(start_ms)
            assert(start_ms < end_ms),"End date must greater than Start Date"
            # faire le test pour verifier que l'on ne prends pas plus de 200 valeurs dans l'interval

    session = HTTP()
    temp=session.get_funding_rate_history(
        category="linear",
        symbol=coin,
        startTime=start_ms,
        endTime= end_ms,
    #limit integer	Limit for data size per page. [1, 200]. Default: 200
    )
    
    df = pd.DataFrame(temp["result"]['list'])
    del temp
    df['fundingRateTimestamp'] = pd.to_numeric(df['fundingRateTimestamp'])
    df['fundingRateTimestamp'] = pd.to_datetime(df['fundingRateTimestamp'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop_duplicates()
    .set_index('fundingRateTimestamp')
    .rename(columns={'annualized_funding': 'annualized_funding_dft'})
    )
    df.index = df.index.floor('min')  # 'T' is shorthand for 'min'

    return df

## OKX

### Historic OKX Rate

In [74]:
import time
import okx.PublicData as PublicData

def get_okx_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    
    if start_ms != None and end_ms != None:
        start_ms = natural_date_to_ms(start_ms)
        end_ms = natural_date_to_ms(end_ms)

    if start_ms != None and end_ms == None:
        start_ms = natural_date_to_ms(start_ms)
        end_ms = start_ms + 7 * 24 * 3600 * 1000

    if start_ms == None and end_ms != None:
        
        end_ms = natural_date_to_ms(end_ms)
        start_ms = end_ms - 7 * 24 * 3600 * 1000
    else:
        #past 7 days
        end_ms = int(time.time() * 1000)
        start_ms = end_ms - 7 * 24 * 3600 * 1000
        

    flag = "0"  # Production trading: 0, Demo trading: 1

    publicDataAPI = PublicData.PublicAPI(flag=flag)

    # Retrieve funding rate history
    temp = publicDataAPI.funding_rate_history(
        instId=coin,
        before=start_ms,
        after=end_ms,
        #limit	String	No	Number of results per request. The maximum is 100; The default is 100
    )
    df = pd.DataFrame(temp['data'])
    del temp
    
    df['fundingTime'] = pd.to_numeric(df['fundingTime'])
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    return df


In [73]:
get_okx_funding_history("ETH-USD-SWAP")
#get_okx_funding_history("BTC-USD-SWAP",start_ms = '2025-06-21') 
#get_okx_funding_history("BTC-USD-SWAP",end_ms='2025-06-27')
#get_okx_funding_history("BTC-USD-SWAP",end_ms='2025-06-27',start_ms = '2025-06-21')

DEBUG:httpcore.connection:connect_tcp.started host='www.okx.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001AB4D444AA0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001AB664CA330> server_hostname='www.okx.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001AB66502030>
DEBUG:httpcore.http2:send_connection_init.started request=<Request [b'GET']>
DEBUG:httpcore.http2:send_connection_init.complete
DEBUG:httpcore.http2:send_request_headers.started request=<Request [b'GET']> stream_id=1
DEBUG:hpack.hpack:Adding (b':method', b'GET') to the header table, sensitive:False, huffman:True
DEBUG:hpack.hpack:Encoding 2 with 7 bits
DEBUG:hpack.hpack:Adding (b':authority', b'www.okx.com') to the header table, sensitive:False, huffman:True
DEBUG:hpack.hpack:

,formulaType,fundingRate,fundingTime,instId,instType,method,realizedRate
0,withRate,0.000031,2025-06-28 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000313541647633
1,withRate,0.000014,2025-06-28 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000135231409159
2,withRate,-0.000029,2025-06-28 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000287968473739
3,withRate,0.000056,2025-06-27 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000560573162987
4,withRate,0.000056,2025-06-27 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000559383318449
5,withRate,-0.000037,2025-06-27 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000366319000835
6,withRate,0.000070,2025-06-26 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000701912910958
7,withRate,0.000057,2025-06-26 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000567331198962
8,withRate,-0.000064,2025-06-26 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000640593288092
9,withRate,-0.000016,2025-06-25 16:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000159787670455


### Current OKX Rate

In [ ]:
import okx.PublicData as PublicData

flag = "0"  # Production trading: 0, Demo trading: 1

publicDataAPI = PublicData.PublicAPI(flag=flag)

# Retrieve funding rate
result = publicDataAPI.get_funding_rate(
    instId="BTC-USD-SWAP",
)

Okx_Funding = pd.DataFrame(result['data'])
Okx_Funding['timestamp'] = pd.to_datetime(Okx_Funding['ts'], unit='ms')
Okx_Funding['fundingRate'] = Okx_Funding['fundingRate'].astype(float)

C:\Users\pmacr\AppData\Local\Temp\ipykernel_23352\723554402.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  Okx_Funding['timestamp'] = pd.to_datetime(Okx_Funding['ts'], unit='ms')


## Hyperliquid

### Hyperliquid history

In [2]:
import requests
import pandas as pd
import time

def get_funding_history(coin: str, start_ms: int, end_ms: int = None):
    payload = {
        "type": "fundingHistory",
        "coin": coin.upper(),
        "startTime": start_ms,
    }
    if end_ms:
        payload["endTime"] = end_ms

    resp = requests.post("https://api.hyperliquid.xyz/info", json=payload)
    resp.raise_for_status()
    data = resp.json()

    df = pd.DataFrame(data)
    if df.empty:
        return df

    df['timestamp'] = pd.to_datetime(df['time'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['premium'] = df['premium'].astype(float)
    return df[['timestamp', 'fundingRate', 'premium']]




## Analysis

In [9]:
binance=get_binance_funding_history('BTCUSD_PERP')
bybi=get_bybit_funding_history("BTCPERP",start_ms="2024-08-01",end_ms="2025-06-30")
merged=binance.join(bybi,how='outer').sort_index()
merged=merged[merged.index> "2024-09-03 00:00:00"]
merged['bin_minus_bybit']=merged['annualized_funding_binance']-merged['annualized_funding_bybit']

DEBUG:root:url: https://dapi.binance.com/dapi/v1/fundingRate
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dapi.binance.com:443
DEBUG:urllib3.connectionpool:https://dapi.binance.com:443 "GET /dapi/v1/fundingRate?symbol=BTCUSD_PERP&limit=1000 HTTP/1.1" 200 None
DEBUG:root:raw response from server:[{"symbol":"BTCUSD_PERP","fundingTime":1722499200001,"fundingRate":"0.00010000","markPrice":"64276.12841843"},{"symbol":"BTCUSD_PERP","fundingTime":1722528000000,"fundingRate":"0.00010000","markPrice":"62828.70000000"},{"symbol":"BTCUSD_PERP","fundingTime":1722556800000,"fundingRate":"0.00010000","markPrice":"65293.05019887"},{"symbol":"BTCUSD_PERP","fundingTime":1722585600000,"fundingRate":"0.00010000","markPrice":"64086.62333147"},{"symbol":"BTCUSD_PERP","fundingTime":1722614400000,"fundingRate":"0.00010000","markPrice":"63339.60000000"},{"symbol":"BTCUSD_PERP","fundingTime":1722643200001,"fundingRate":"0.00010000","markPrice":"61387.80000000"},{"symbol":"BTCUSD_PERP","fundi

In [10]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'  # Open in browser for full interactivity

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['bin_minus_bybit'],
    mode='lines',
    name='Binance - Bybit',
    line=dict(color='royalblue')
))

fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.update_layout(
    title='Funding Rate Difference: Binance - Bybit',
    xaxis_title='Date',
    yaxis_title='Annualized Funding Rate Difference',
    template='plotly_white',
    hovermode='x unified',
    dragmode='zoom',  # Allows zooming both horizontally and vertically
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
        fixedrange=False  # allow x-axis zoom
    ),
    yaxis=dict(
        fixedrange=False  # allow y-axis zoom
    )
)

fig.show()


In [ ]:
import requests
requests.get("https://fapi.binance.com/fapi/v1/premiumIndex?symbol=BTCUSDT").json()
requests.get("https://api-testnet.bybit.com/v5/market/tickers?category=linear&symbol=BTCUSDT").json()

In [11]:
window = 20
n_std = 1.5

# Calculate the rolling mean and standard deviation of the merged['bin_minus_bybit']
merged["rolling_mean"] = merged['bin_minus_bybit'].rolling(window=30).mean()
merged["rolling_std"] = merged['bin_minus_bybit'].rolling(window=30).std()

# Calculate the z-score (number of standard deviations away from the rolling mean)
zscore = (merged['bin_minus_bybit'] - merged["rolling_mean"]) / merged["rolling_std"]

upper_band = merged["rolling_mean"] + n_std * merged["rolling_std"]
lower_band = merged["rolling_mean"] - n_std * merged["rolling_std"]

In [20]:
import plotly.graph_objects as go
import plotly.io as pio

# Force Plotly to open in the browser
pio.renderers.default = "browser"

# Parameters
window = 30
n_std = 1.5

# Calculate rolling stats
merged['rolling_mean'] = merged['bin_minus_bybit'].rolling(window=window).mean()
merged['rolling_std'] = merged['bin_minus_bybit'].rolling(window=window).std()

# Z-score
zscore = (merged['bin_minus_bybit'] - merged['rolling_mean']) / merged['rolling_std']

# Bands
upper_band = merged['rolling_mean'] + n_std * merged['rolling_std']
lower_band = merged['rolling_mean'] - n_std * merged['rolling_std']

# Start plot
fig = go.Figure()

# Funding spread
fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['bin_minus_bybit'],
    mode='lines',
    name='Funding Rate Spread',
    line=dict(color='red')
))

# Rolling mean
fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['rolling_mean'],
    mode='lines',
    name='Rolling Mean',
    line=dict(color='green', dash='dot')
))

# Upper and lower bands
fig.add_trace(go.Scatter(
    x=merged.index,
    y=upper_band,
    mode='lines',
    name='Upper Band',
    line=dict(color='blue', width=1),
    showlegend=True
))
fig.add_trace(go.Scatter(
    x=merged.index,
    y=lower_band,
    mode='lines',
    name='Lower Band',
    line=dict(color='blue', width=1),
    fill='tonexty',  # fills between this and previous trace
    fillcolor='rgba(173, 216, 230, 0.2)',
    showlegend=True
))

# Optional: Outlier dots where z-score > 1.5
# threshold = 1.5
# outliers = merged[np.abs(zscore) > threshold]
# fig.add_trace(go.Scatter(
#     x=outliers.index,
#     y=outliers['bin_minus_bybit'],
#     mode='markers',
#     name='Z > 1.5',
#     marker=dict(color='black', size=6, symbol='circle-open')
# ))

# Final layout
fig.update_layout(
    title='Funding Rate Spread: Binance - Bybit (With Rolling Mean & ±1.5 Std Bands)',
    xaxis_title='Date',
    yaxis_title='Annualized Funding Rate Difference',
    template='plotly_white',
    hovermode='x unified',
    dragmode='zoom',
    xaxis=dict(
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ]
        )
    ),
    yaxis=dict(fixedrange=False)
)

fig.show()


##  Get rates from RHO API

In [9]:
DES="""[
    {
        "descriptor": {
            "id": "0x852d33076c184e71c510bd29bb2e8ad041f4e32ebfb4dd119469332664a56bce",
            "sourceName": "Binance",
            "instrumentName": "BTCUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0xded289a6f761ddee825f915048efadefcbb1d3d9c11d36c793558192eeb2f75f",
                "marketId": "0x852d33076c184e71c510bd29bb2e8ad041f4e32ebfb4dd119469332664a56bce",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x81801f4f2e9dd9"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x02c6222aa3fe"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0xf9d1aa22d78925efc9850a38dea62b894b2609a563e21f22e7321c0fec507872",
                "marketId": "0x852d33076c184e71c510bd29bb2e8ad041f4e32ebfb4dd119469332664a56bce",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xab8b081409c379"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x022ecb25bff6"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x04f4ed5063f4"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0x5dc9814bc6650ce1e620667427cea9497265edd04844c865aaa9e49faf7fe1e0",
            "sourceName": "Binance",
            "instrumentName": "ETHUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0xd3aa0e4606a0f89e3af376f5492f38f527b3feac9c51e2b1bc46685d9cd7c16b",
                "marketId": "0x5dc9814bc6650ce1e620667427cea9497265edd04844c865aaa9e49faf7fe1e0",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x9186bb8e94a153"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x02dd6aa18bfe"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0xd64e23ced32b015386d436971ebbe0d277108519c99cd31d8ca146636601ec4f",
                "marketId": "0x5dc9814bc6650ce1e620667427cea9497265edd04844c865aaa9e49faf7fe1e0",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xaae96cf8e9dd1a"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x021e7ed283f4"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x04fbe9740ff2"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0xfe4004698d306fdf8a2654e9bb8e2d9a101b80fea1c61bcf93483f748fe06d62",
            "sourceName": "OKX",
            "instrumentName": "BTCUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0x768b2bc192e40a83f6115a07654d2f6774f6a6cf5267c32ef8efc1a7620d835b",
                "marketId": "0xfe4004698d306fdf8a2654e9bb8e2d9a101b80fea1c61bcf93483f748fe06d62",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x99ff0914bdd6b6"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x032ee841b7f7"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0xea33893797a99df16a7241d62a4bc5dd6ff8ac1a9f96b3b34973b1081776d021",
                "marketId": "0xfe4004698d306fdf8a2654e9bb8e2d9a101b80fea1c61bcf93483f748fe06d62",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xa5ad962c4111ba"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x02003a37eff7"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x052f2279a7ee"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0x123fa87100b42b4127e5d4bae4f65b60ea48015b4065690cf6b5f6c9aa7316f5",
            "sourceName": "OKX",
            "instrumentName": "ETHUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0x9db316545a09382dc73e6de25b36a4a938eb08b3a69e08f48bb1df0a21a00f42",
                "marketId": "0x123fa87100b42b4127e5d4bae4f65b60ea48015b4065690cf6b5f6c9aa7316f5",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x95cd3b458a5d1c"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x025fb01f73fd"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0x8bcaaecdad8b53a19a2e7c34ab3a02c3004a9acaae5697c73d5024fc47f1d1c2",
                "marketId": "0x123fa87100b42b4127e5d4bae4f65b60ea48015b4065690cf6b5f6c9aa7316f5",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x9f4780360b7aff"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x016fdf56b7f9"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x03cf8f762bf6"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0x46755f2b1f6a1f5d06bed293d7d87546ec199b2b4bc2bacc0b8774f441cf68ac",
            "sourceName": "CoinDesk Indices",
            "instrumentName": "CESR",
            "tag": "staking",
            "version": 1,
            "underlying": "0x82aF49447D8a07e3bd95BD0d56f35241523fBab1",
            "underlyingName": "WETH",
            "underlyingDecimals": 18,
            "underlyingIsWrappedNativeToken": true,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0x96dcd29de995884fe8c0d2723c2d1f2b364819d9b35d7f5840a7405f43e868a3",
                "marketId": "0x46755f2b1f6a1f5d06bed293d7d87546ec199b2b4bc2bacc0b8774f441cf68ac",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x9b66d0"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x058d15e176280000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x6dbbd8a3615e90"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x058d15e176280000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x038d7ea4c68000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0320"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x1ba5abf9e77937fff4"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x1ba5abf9e77937fff4"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x038d7ea4c68000"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x2386f26fc10000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x58d15e17628000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x5af3107a4000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x09184e72a000"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0110d9316ec000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x00"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0xc1c48702b028d547c0892b65f69652107d39d6fa5363aa4c19539b1af20bff90",
            "sourceName": "Bybit",
            "instrumentName": "BTCUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0x177c9d3564d81516826246c8302c68913f1e139716d2f20beccd849aac2c8c98",
                "marketId": "0xc1c48702b028d547c0892b65f69652107d39d6fa5363aa4c19539b1af20bff90",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xa9b67ddde1552d"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x02aa319bf3f6"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0x2a5178b3eb6f43f52ab0bbd9d67f55f8bb4cfd901c1d1b259654d9dab2fdb226",
                "marketId": "0xc1c48702b028d547c0892b65f69652107d39d6fa5363aa4c19539b1af20bff90",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xb4b6bbb1df9a1b"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x017dd79e1000"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x0428093a03f6"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    },
    {
        "descriptor": {
            "id": "0x93e1a736148ecf05b49a78d8538b7d64f6c61a6ff1f60a39cc55dd8d7ba12ef9",
            "sourceName": "Bybit",
            "instrumentName": "ETHUSDT Funding",
            "tag": "funding",
            "version": 1,
            "underlying": "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9",
            "underlyingName": "USD₮0",
            "underlyingDecimals": 6,
            "underlyingIsWrappedNativeToken": false,
            "rateMathType": 0
        },
        "futures": [
            {
                "id": "0xd940594b3ffc8a076dd5566dae88ca17f5b638f97ea1ad52350accaeac25c5d0",
                "marketId": "0x93e1a736148ecf05b49a78d8538b7d64f6c61a6ff1f60a39cc55dd8d7ba12ef9",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x6812ab70"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x708e10"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0x99bad8e98d711e"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x02b381cb83f4"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            },
            {
                "id": "0xdec6434eace5a131d6f2243b2485038493a02063e7211b008352b35395854244",
                "marketId": "0x93e1a736148ecf05b49a78d8538b7d64f6c61a6ff1f60a39cc55dd8d7ba12ef9",
                "termStart": {
                    "type": "BigNumber",
                    "hex": "0x683b89f0"
                },
                "termLength": {
                    "type": "BigNumber",
                    "hex": "0x75d410"
                },
                "vAMMParams": {
                    "lowerBoundRate": {
                        "type": "BigNumber",
                        "hex": "-0x0de0b6b3a7640000"
                    },
                    "currentFutureRate": {
                        "type": "BigNumber",
                        "hex": "0xc060a8d7c801e7"
                    },
                    "upperBoundRate": {
                        "type": "BigNumber",
                        "hex": "0x0de0b6b3a7640000"
                    },
                    "intervalLength": {
                        "type": "BigNumber",
                        "hex": "0x11c37937e08000"
                    },
                    "intervalsCount": {
                        "type": "BigNumber",
                        "hex": "0x0190"
                    }
                },
                "totalLiquidityNotional": {
                    "type": "BigNumber",
                    "hex": "0x01bcb4df1bf8"
                },
                "openInterest": {
                    "type": "BigNumber",
                    "hex": "0x00"
                },
                "pauseConfiguration": {
                    "trade": false,
                    "liquidityProvision": false,
                    "liquidityRemoval": false,
                    "liquidation": false,
                    "positionsOwnershipTransfer": false
                }
            }
        ],
        "openInterest": {
            "type": "BigNumber",
            "hex": "0x00"
        },
        "totalLiquidityNotional": {
            "type": "BigNumber",
            "hex": "0x047036aa9fec"
        },
        "riskParameters": {
            "minNotional": {
                "type": "BigNumber",
                "hex": "0x0f4240"
            },
            "liquidationThresholdDelta": {
                "type": "BigNumber",
                "hex": "0xd529ae9e860000"
            },
            "marginThresholdDelta": {
                "type": "BigNumber",
                "hex": "0x013fbe85edc90000"
            },
            "marginThresholdTolerance": {
                "type": "BigNumber",
                "hex": "0x01c6bf52634000"
            },
            "maxRateImpactPerTrade": {
                "type": "BigNumber",
                "hex": "0x2aa1efb94e0000"
            },
            "hedgeMarginFactor": {
                "type": "BigNumber",
                "hex": "0x03782dace9d90000"
            },
            "marginRequirementSecondsFloor": 1814400,
            "liquidationMarginRequirementSecondsFloor": 1814400,
            "maturityLockoutSeconds": 3600
        },
        "rewardParameters": {
            "transferOwnershipReward": {
                "type": "BigNumber",
                "hex": "0x06f05b59d3b20000"
            },
            "positionLiquidationReward": {
                "type": "BigNumber",
                "hex": "0x0429d069189e0000"
            },
            "provisionCancellationReward": {
                "type": "BigNumber",
                "hex": "0x11c37937e08000"
            },
            "lpPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x016345785d8a0000"
            },
            "lpNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x0775f05a074000"
            },
            "minLpFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            },
            "protocolPriceImpactFeeFactor": {
                "type": "BigNumber",
                "hex": "0x00"
            },
            "protocolNotionalFeeFactor": {
                "type": "BigNumber",
                "hex": "0x03328b944c4000"
            },
            "minProtocolFee": {
                "type": "BigNumber",
                "hex": "0x2710"
            }
        },
        "pauseConfiguration": {
            "withdraw": false,
            "deposit": false
        }
    }
]"""

In [3]:
import json

In [ ]:
data=json.loads(DES) #DES being issued from the  browser console

# Extract and prepare DataFrames
descriptions_df = pd.DataFrame([
    {
        "market": d["descriptor"]["sourceName"] + d["descriptor"]["instrumentName"],
        "id": d["descriptor"]["id"]
    }
    for d in data
])

futures_df = pd.DataFrame([
    {
        "id": f["id"],
        "marketId": f["marketId"]
    }
    for d in data for f in d["futures"]
])

# Merge on market ID
futures_df = futures_df.merge(
    descriptions_df,
    left_on="marketId",
    right_on="id",
    how="left"
).drop(columns=["marketId", "id_y"]).rename(columns={"id_x": "id"})

# Optional: Reorder columns if needed
# futures_df = futures_df[["id", "market"]]


# Define the data
maturity_data = [
    ("0xded289a6f761ddee825f915048efadefcbb1d3d9c11d36c793558192eeb2f75f", "2025-07-25"),
    ("0xf9d1aa22d78925efc9850a38dea62b894b2609a563e21f22e7321c0fec507872", "2025-08-29"),
    ("0xd3aa0e4606a0f89e3af376f5492f38f527b3feac9c51e2b1bc46685d9cd7c16b", "2025-07-25"),
    ("0xd64e23ced32b015386d436971ebbe0d277108519c99cd31d8ca146636601ec4f", "2025-08-29"),
    ("0x768b2bc192e40a83f6115a07654d2f6774f6a6cf5267c32ef8efc1a7620d835b", "2025-07-25"),
    ("0xea33893797a99df16a7241d62a4bc5dd6ff8ac1a9f96b3b34973b1081776d021", "2025-08-29"),
    ("0x9db316545a09382dc73e6de25b36a4a938eb08b3a69e08f48bb1df0a21a00f42", "2025-07-25"),
    ("0x8bcaaecdad8b53a19a2e7c34ab3a02c3004a9acaae5697c73d5024fc47f1d1c2", "2025-08-29"),
    ("0x96dcd29de995884fe8c0d2723c2d1f2b364819d9b35d7f5840a7405f43e868a3", "2025-09-26"),
    ("0x177c9d3564d81516826246c8302c68913f1e139716d2f20beccd849aac2c8c98", "2025-07-25"),
    ("0x2a5178b3eb6f43f52ab0bbd9d67f55f8bb4cfd901c1d1b259654d9dab2fdb226", "2025-08-29"),
    ("0xd940594b3ffc8a076dd5566dae88ca17f5b638f97ea1ad52350accaeac25c5d0", "2025-07-25"),
    ("0xdec6434eace5a131d6f2243b2485038493a02063e7211b008352b35395854244", "2025-08-29"),
]

# Create DataFrame
maturity_df = pd.DataFrame(maturity_data, columns=["id", "maturity"])

# Optionally convert 'maturity' to datetime
maturity_df["maturity"] = pd.to_datetime(maturity_df["maturity"])

# Merge on 'id' to add maturity to futures_df
futures_df = futures_df.merge(maturity_df, on="id", how="left")
# Format the dictionary

futures_dict = {
    f"{row['market'].replace(" ", "_")}_{row['maturity'].strftime('%b').replace(" ", "_").replace(".", "")}_{row['maturity'].day}": row["id"]
    for _, row in futures_df.iterrows()
}

futures_df.to_csv('futures_df.csv')


                                                  id                  market
0  0xded289a6f761ddee825f915048efadefcbb1d3d9c11d...  BinanceBTCUSDT Funding
1  0xf9d1aa22d78925efc9850a38dea62b894b2609a563e2...  BinanceBTCUSDT Funding
2  0xd3aa0e4606a0f89e3af376f5492f38f527b3feac9c51...  BinanceETHUSDT Funding
3  0xd64e23ced32b015386d436971ebbe0d277108519c99c...  BinanceETHUSDT Funding
4  0x768b2bc192e40a83f6115a07654d2f6774f6a6cf5267...      OKXBTCUSDT Funding


In [61]:
int(natural_date_to_ms("2025-07-02")/1000)

1751407200

In [75]:
FUTURES = {'BinanceBTCUSDT_Funding_juil_25': '0xded289a6f761ddee825f915048efadefcbb1d3d9c11d36c793558192eeb2f75f',
 'BinanceBTCUSDT_Funding_août_29': '0xf9d1aa22d78925efc9850a38dea62b894b2609a563e21f22e7321c0fec507872',
 'BinanceETHUSDT_Funding_juil_25': '0xd3aa0e4606a0f89e3af376f5492f38f527b3feac9c51e2b1bc46685d9cd7c16b',
 'BinanceETHUSDT_Funding_août_29': '0xd64e23ced32b015386d436971ebbe0d277108519c99cd31d8ca146636601ec4f',
 'OKXBTCUSDT_Funding_juil_25': '0x768b2bc192e40a83f6115a07654d2f6774f6a6cf5267c32ef8efc1a7620d835b',
 'OKXBTCUSDT_Funding_août_29': '0xea33893797a99df16a7241d62a4bc5dd6ff8ac1a9f96b3b34973b1081776d021',
 'OKXETHUSDT_Funding_juil_25': '0x9db316545a09382dc73e6de25b36a4a938eb08b3a69e08f48bb1df0a21a00f42',
 'OKXETHUSDT_Funding_août_29': '0x8bcaaecdad8b53a19a2e7c34ab3a02c3004a9acaae5697c73d5024fc47f1d1c2',
 'CoinDesk_IndicesCESR_sept_26': '0x96dcd29de995884fe8c0d2723c2d1f2b364819d9b35d7f5840a7405f43e868a3',
 'BybitBTCUSDT_Funding_juil_25': '0x177c9d3564d81516826246c8302c68913f1e139716d2f20beccd849aac2c8c98',
 'BybitBTCUSDT_Funding_août_29': '0x2a5178b3eb6f43f52ab0bbd9d67f55f8bb4cfd901c1d1b259654d9dab2fdb226',
 'BybitETHUSDT_Funding_juil_25': '0xd940594b3ffc8a076dd5566dae88ca17f5b638f97ea1ad52350accaeac25c5d0',
 'BybitETHUSDT_Funding_août_29': '0xdec6434eace5a131d6f2243b2485038493a02063e7211b008352b35395854244'}

In [ ]:
import requests

INTERVAL_SECONDS = {
    "1m": 60,
    "5m": 5 * 60,
    "15m": 15 * 60,
    "30m": 30 * 60,
    "1h": 60 * 60,
    "4h": 4 * 60 * 60,
    "1d": 24 * 60 * 60,
    "1w": 7 * 24 * 60 * 60,
}

def fetch_candles(future_id -> str, start-> int, end -> int, interval="1h", max_candles=350 )-> pd.DataFrame:
    """_summary_

    Args:
        future_id (_type_): _description_
        start (_type_): _description_
        end (_type_): _description_
        interval (str, optional): _description_. Defaults to "1h".
        max_candles (int, optional): _description_. Defaults to 350.

    Raises:
        ValueError: _description_

    Returns:
        pd.DataFrame: _description_
    """

    if interval not in INTERVAL_SECONDS: # type: ignore
        raise ValueError(f"Unsupported interval: {interval}")

    step = INTERVAL_SECONDS[interval] * max_candles # type: ignore
    candles = []
    current = start

    while current < end:
        next_time = min(current + step, end)
        url = (
            f"https://ds.rhoservice.com/api/v1/market-data/candles"
            f"?futureId={future_id}&from={current}&to={next_time}&interval={interval}"
        )
        response = requests.get(url).json()

        if 'error' in response:
            print("Error:", response)
            break

        candles.extend(response.get("candles", []))
        current = next_time

    decimal = 16
    # Apply transformations
    candles=pd.DataFrame(candles)
    candles["close"] = (pd.to_numeric(candles["close"]) / 10**16).round(4)
    candles["high"] = (pd.to_numeric(candles["high"]) / 10**16).round(4)
    candles["low"] = (pd.to_numeric(candles["low"]) / 10**16).round(4)
    candles["open"] = (pd.to_numeric(candles["open"]) / 10**16).round(4)
    candles["volume"] = (pd.to_numeric(candles["volume"]) / 10**decimal).round(4)

    # Convert the datetime string column to datetime dtype with UTC awareness
    candles['timestamp'] = pd.to_datetime(candles['timestamp'], utc=True)

    # Set the datetime column as the index
    candles = candles.set_index('timestamp')


    

    return candles



In [ ]:
abv=fetch_candles(futures_dict['BinanceBTCUSDT_Funding_août_29'],1743458400,1751407200)
#  1743458400  1er Avril minuit

In [84]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Candlestick(
        x=abv.index,
        open=abv["open"],
        high=abv["high"],
        low=abv["low"],
        close=abv["close"],
        increasing_line_color='green',
        decreasing_line_color='red'
    )
])

fig.update_layout(
    title="Candlestick Chart",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False
)

fig.show()


In [51]:
import plotly.express as px

fig = px.line(a, x=a.index, y="close", title="Close Price Over Time")
fig.update_traces(line_color='blue')
fig.show()


In [ ]:
import requests
requests.get("https://fapi.binance.com/fapi/v1/premiumIndex?symbol=BTCUSDT").json()
requests.get("https://api-testnet.bybit.com/v5/market/tickers?category=linear&symbol=BTCUSDT").json()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fapi.binance.com:443
DEBUG:urllib3.connectionpool:https://fapi.binance.com:443 "GET /fapi/v1/premiumIndex?symbol=BTCUSDT HTTP/1.1" 200 None


{'symbol': 'BTCUSDT',
 'markPrice': '107761.60000000',
 'indexPrice': '107810.44891304',
 'estimatedSettlePrice': '107727.56675715',
 'lastFundingRate': '0.00000672',
 'interestRate': '0.00010000',
 'nextFundingTime': 1751299200000,
 'time': 1751286044000}

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api-testnet.bybit.com:443
DEBUG:urllib3.connectionpool:https://api-testnet.bybit.com:443 "GET /v5/market/tickers?category=linear&symbol=BTCUSDT HTTP/1.1" 200 738


{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'lastPrice': '104418.70',
    'indexPrice': '107609.37',
    'markPrice': '104418.70',
    'prevPrice24h': '103711.00',
    'price24hPcnt': '0.006823',
    'highPrice24h': '104888.00',
    'lowPrice24h': '101400.00',
    'prevPrice1h': '104767.50',
    'openInterest': '837113.34',
    'openInterestValue': '87410286715.46',
    'turnover24h': '66114244.7851',
    'volume24h': '636.5620',
    'fundingRate': '-0.005',
    'nextFundingTime': '1751299200000',
    'predictedDeliveryPrice': '',
    'basisRate': '',
    'deliveryFeeRate': '',
    'deliveryTime': '0',
    'ask1Size': '5.446',
    'bid1Price': '104266.70',
    'ask1Price': '104418.70',
    'bid1Size': '0.002',
    'basis': '',
    'preOpenPrice': '',
    'preQty': '',
    'curPreListingPhase': ''}]},
 'retExtInfo': {},
 'time': 1751288868761}